In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

data = pd.read_excel(r'I:\1\DataSet.xlsx')

X = data.drop(columns=["V"])
y = data["V"]  

imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

X_imputed[np.isinf(X_imputed)] = np.nan
X_imputed = imputer.fit_transform(X_imputed)

X = pd.DataFrame(X_imputed, columns=X.columns)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = np.expand_dims(X_scaled, axis=2)

def create_cnn_model():
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_scaled.shape[1], 1)),
        Dropout(0.2),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

estimator = KerasRegressor(build_fn=create_cnn_model, verbose=0)

param_grid = {'batch_size': [16, 32, 64],
              'epochs': [100, 200, 300]
              'validation_split': [0.1, 0.2, 0.3]
              'verbose': [0, 1, 2]}

kf = KFold(n_splits=10, shuffle=True)

grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error')
grid_result = grid_search.fit(X_scaled, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

best_model = grid_result.best_estimator_

best_params_df = pd.DataFrame(grid_result.best_params_, index=[0])
best_params_df.to_excel(r'I:\1\CNN-optimal parameter.xlsx', index=False)

In [ ]:
metrics = {'Fold': [], 'RMSE': [], 'R^2': []}
fold = 1

for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    y_train = y_train.values
    y_test = y_test.values

    history = best_model.fit(X_train, y_train, epochs=best_params_df['epochs'].iloc[0], batch_size=best_params_df['batch_size'].iloc[0], verbose=0)

    y_pred = best_model.predict(X_test)
    y_pred = np.array(y_pred)  
    y_pred = np.squeeze(y_pred)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    metrics['Fold'].append(fold)
    metrics['RMSE'].append(rmse)
    metrics['R^2'].append(r2)


    print(f"Fold {fold}: RMSE={rmse}, R^2={r2}")

    fold += 1

metrics_df = pd.DataFrame(metrics)
metrics_df.to_excel(r'I:\1\CNN-Ten-fold cross validation performance metric.xlsx', index=False)